In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import subprocess
import pandas as pd
import openai
import nest_asyncio
import asyncio

nest_asyncio.apply()

# Set OpenAI API key
with open('/content/drive/MyDrive/key.txt', 'r') as file:
    api_key = file.read().strip()
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = api_key

# Paths
folder_path = '/content/drive/MyDrive/Programs/'
csv_input_path = os.path.join(folder_path, 'program_rmse_results.csv')
results_df = pd.read_csv(csv_input_path)
program_number = len(results_df) + 1
new_prompt_filename = f"program{program_number}.txt"
new_code_filename = f"program{program_number}.py"

# Best previous prompt
best_prompt_file = results_df.iloc[0]['prompt']
prompt_path = os.path.join(folder_path, best_prompt_file)
with open(prompt_path, 'r', encoding='utf-8') as file:
    original_prompt = file.read()

# Async functions to interact with ChatGPT
async def get_improved_prompt(original_prompt):
    enhanced_instruction = (
        f"Improve this prompt:\n{original_prompt}\n"
        "Explicitly instruct the generated code to format all numerical outputs (RMSE, R², etc.) in scientific notation using '{:.2e}'.format().\n"
        "Explicitly instruct to use this correct dataset URL:\n"
        "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/main/SO2TONS_dataset.csv"
    )
    response = await openai.AsyncOpenAI().chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": enhanced_instruction}],
        temperature=0.2
    )
    return response.choices[0].message.content

async def generate_code(prompt):
    instruction = (
        f"{prompt}\nGenerate ONLY valid Python code. No explanations. "
        "DO NOT include markdown, backticks, or code block markers (```). "
        "Explicitly include at the beginning:\n"
        "import subprocess\nsubprocess.check_call(['pip', 'install', '--upgrade', 'scikit-learn'])\n"
        "import numpy as np\n"
        "Use np.sqrt(mean_squared_error(y_test, y_pred)) to calculate RMSE explicitly."
    )
    response = await openai.AsyncOpenAI().chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": instruction}],
        temperature=0.2
    )
    return response.choices[0].message.content.strip('` \n')

async def correct_code(buggy_code, error_message):
    instruction = (
        "The following Python code generated an error:\n"
        f"Error: {error_message}\n"
        f"Code:\n{buggy_code}\n"
        "Correct the code and provide ONLY the fixed Python code. DO NOT include markdown, backticks, or code block markers (```). "
        "Explicitly include at the beginning:\n"
        "import subprocess\nsubprocess.check_call(['pip', 'install', '--upgrade', 'scikit-learn'])\n"
        "import numpy as np\n"
        "Use np.sqrt(mean_squared_error(y_test, y_pred)) to calculate RMSE explicitly."
    )
    response = await openai.AsyncOpenAI().chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": instruction}],
        temperature=0.2
    )
    return response.choices[0].message.content.strip('` \n')

# Execute Python code and capture errors
def execute_and_test(filepath):
    try:
        result = subprocess.run(["python", filepath], capture_output=True, text=True, timeout=300)
        output = result.stdout + result.stderr
        if result.returncode == 0:
            print(f"Execution successful. Output:\n{output}")
            return True, output
        else:
            print(f"Execution failed with error:\n{output}")
            return False, output
    except subprocess.TimeoutExpired:
        return False, "Execution timed out."

# Main loop
async def main_loop(max_trials=3):
    improved_prompt = await get_improved_prompt(original_prompt)

    # Save improved prompt
    with open(os.path.join(folder_path, new_prompt_filename), 'w', encoding='utf-8') as file:
        file.write(improved_prompt)
    print(f"Prompt saved to {new_prompt_filename}")

    trial = 1
    success = False
    code = await generate_code(improved_prompt)

    while trial <= max_trials and not success:
        code_path = os.path.join(folder_path, new_code_filename)
        with open(code_path, 'w', encoding='utf-8') as file:
            file.write(code)
        print(f"Trial {trial}: Code saved to {new_code_filename}, attempting execution...")

        success, output = execute_and_test(code_path)

        if success and 'rmse' in output.lower():
            print(f"Trial {trial}: Program executed successfully with RMSE output.")
            break
        elif not success:
            print(f"Trial {trial}: Error detected, submitting to ChatGPT for correction...")
            code = await correct_code(code, output)

        trial += 1

    if success:
        print(f"Code executed successfully after {trial} trial(s).")
    else:
        print(f"Failed to execute successfully after {max_trials} trials.")

# Run the loop
asyncio.run(main_loop(max_trials=5))

Mounted at /content/drive
Prompt saved to program7.txt
Trial 1: Code saved to program7.py, attempting execution...
Execution successful. Output:
RMSE: 1.02e-05
R² Score: 0.48
Final RMSE: 1.02e-05

Trial 1: Program executed successfully with RMSE output.
Code executed successfully after 1 trial(s).
